<a href="https://colab.research.google.com/github/liuy01510/portfolio/blob/master/Machine_Learning_I_Data_Preprocessing_PF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction
- Import data from the source and perform data cleaning to prepare it for machine learning.
- Data source: [Median Rent by Town and Flat Type](https://data.gov.sg/dataset/median-rent-by-town-and-flat-type)


# Importing required data

## Modules importing

In [0]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import collections as coll
import requests
import zipfile
import json

## Data Import Function

In [0]:
def Data_Import():
    url='https://data.gov.sg/dataset/b35046dc-7428-4cff-968d-ef4c3e9e6c99/download' # data url
    req=requests.get(url) # get the url
    res=req.content # get the content of the response

    zipFileLoc='/content/Zipped_Rent_Data' # zip file location
    fileLoc='/content/Rent_Data' # data location

    ## Saving the zip file
    with open(zipFileLoc,'wb') as f:
        f.write(res) # saving the zip file
    
    ## Extracting the data from the zip file
    with zipfile.ZipFile(zipFileLoc) as f:
        f.extractall(path=fileLoc)
    
    ## Writing the CSV file into a dataframe
    csvName='/median-rent-by-town-and-flat-type.csv'
    result=pd.read_csv(fileLoc+csvName)
    return result


In [0]:
# Storing the raw data into a dataframe
all_tables=coll.OrderedDict()
csvName='/content/Rent_Data/median-rent-by-town-and-flat-type.csv'

try:
    all_tables['raw']=pd.read_csv(csvName)
except:
    Data_Import()
    all_tables['raw']=pd.read_csv(csvName)

# Data cleaning

## Preliminary data cleaning

In [0]:
# Dropping all entries with 'na' values in the 'median_rent' column
table=all_tables['raw']
table=table[[True if i!='na' and i!='-' else False for i in table['median_rent']]]

# Splitting out the quarter column to individual year and quarter columns
table=table.assign(Quarter=[int(i.split('-')[1][1]) for i in table['quarter']])
table=table.assign(Year=[int(i.split('-')[0]) for i in table['quarter']])
table=table.drop(columns=['quarter'])

# Reindexing the table
table=table.reset_index().drop(columns=['index'])

# Correcting the median_rent type to float
table.loc[:,'median_rent']=table.loc[:,'median_rent'].apply(int)
all_tables['raw']=table

# Correcting for the price index
- The indicative prices of the different years is an uneven baseline due to the occurence of inflation that happens across the years.
- The HDB resale price index will be used to approximate this inflation.

In [0]:
# Loading the json data from the index into a dict.
def Get_Price_Index():
    url2='https://data.gov.sg/api/action/datastore_search?resource_id=52e93430-01b7-4de0-80df-bc83d0afed40&limit=10000' # API url

    req=requests.get(url2)
    res=req.content

    result=json.loads(res)
    result=result['result']['records']
    return result

try:
    pIndex
except:
    pIndex=Get_Price_Index()

In [0]:
# Extracting the useful data
index=coll.OrderedDict()
for x in pIndex:
    y=x['quarter'].split('-')
    index[(int(y[1][1]),int(y[0]))]=float(x['index'])

In [230]:
# Performing the index correction
v=coll.OrderedDict() # dict to store important variable values

## Base Index
table=all_tables['raw']
x=set([(q,y) for q,y in zip(table['Quarter'],table['Year'])])
x=sorted(x,key=lambda y:y[0],reverse=True) # sorting according to quarter
x=sorted(x,key=lambda y:y[1],reverse=True) # sorting according to year
v['Base Index']=index[x[0]] # latest year index (Q1,2020)

## Applying index correction
grouped=table.groupby(['Quarter','Year'])
for n,t in grouped:
    cf=v['Base Index']/index[n]
    table.loc[t.index,'median_rent']=table.loc[t.index,'median_rent']*cf

all_tables['raw']=table
all_tables['raw']

,town,flat_type,median_rent,Quarter,Year
0,ANG MO KIO,3-RM,1431.292517,2,2005
1,ANG MO KIO,4-RM,1699.659864,2,2005
2,BEDOK,3-RM,1431.292517,2,2005
3,BEDOK,4-RM,1610.204082,2,2005
4,BEDOK,5-RM,1833.843537,2,2005
...,...,...,...,...,...
4347,WOODLANDS,EXEC,2000.000000,1,2020
4348,YISHUN,3-RM,1600.000000,1,2020
4349,YISHUN,4-RM,1800.000000,1,2020
4350,YISHUN,5-RM,2030.000000,1,2020


- The table above shows the table of data.
- However, this table must still undergo further data processing to be suitable for ML.